In [4]:
!pip install sqlalchemy

In [5]:
import SQLAlchemy

ModuleNotFoundError: No module named 'SQLAlchemy'

In [23]:
import os
import pyodbc
import pandas as pd

In [101]:
path = 'C:/Users/arthur.silva/source/repos/LASOS-INT/TTBsim/01_Config/DBFiles/ToolBoxMix_Development - Faz Tudo Quero Logo.mdb'
conn_str =     conn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};' +
            'DBQ='+f'{path};')
print(conn_str)
conn = pyodbc.connect(conn_str)

DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:/Users/arthur.silva/source/repos/LASOS-INT/TTBsim/01_Config/DBFiles/ToolBoxMix_Development - Faz Tudo Quero Logo.mdb;


In [27]:
cursor = conn.cursor()
for i in cursor.tables(tableType='TABLE'):
    print(i.table_name)

ACABADO
AGRU_ENTID
AGRU_GRUPO
AGRU_LINK
ALTERNATIVAS
ATIV_CENTRO
ATIV_PRECED
ATIVP
CAL_EXCE
CAL_PART
COMPRADO
CONJUGADAS
CONJUNTOS
CTRL_APONTA
CTRL_ESTOQ_DEPO
CTRL_ESTOQ_MOV
CTRL_ESTOQ_POS
CTRL_MARCO
DEC_CRITERIOS_DE_SEQUENCIAMENTO
DEC_DEDICACAO_MAQUINAS
DEC_FLEXIBILIDADES
DEC_HORA_EXTRA
DEC_HORARIOS
DEC_LOTES_PRODUCAO
DEC_LOTES_TRANSFERENCIA
DEC_PLANO_COMPRAS
DEC_PRAZOS_ENTREGAS
DEC_PRIORIDADE
DEC_RESTRICAO_INICIO
DEC_SIMULAR_ORDEM_PRODUCAO
DEC_SPLITS
DEC_SUBCONTRATACOES
DEC_TEMPO_TRANS
ELO_PERDIDO
Erros ao colar
ESCALARES
ESTOQUES
ESTOQUES_PROCESSO
EXT_COMPRA
EXT_COMPRA_RECEB
EXT_EMPRESA
EXT_PLANO_MESTRE
EXT_VENDA
EXT_VENDA_ENTREGA
EXT_VENDA_STATUS
FABRICA
FABRICA_OLD
FERRAMENTA
FLEX_ESCOPO_MAQ
FLEX_ESCOPO_OPER
HORARIOS
HORARIOS_PARTICAO
ITEM
ITEM_ESTRU
ITENS_NAVEGACAO
ITENS_SELECAO
LISTA_APLICACOES
LISTA_APLICACOES_AUX
MAQUINA
OPERACOES
REL_ANALISE_FINANCEIRA
REL_DEBUG
REL_DEMANDAS_LIQUIDAS
REL_ENTREGAS_NAO_REALIZADAS
REL_NECESS_MAT
REL_PONTUALIDADE
REL_RECEB_MAT
REL_SEQUENCIA_DETAL

In [99]:
alter = pd.read_sql('select * from ALTERNATIVAS', conn)

jobs = alter['CdItem'].unique()
machines = alter['CdMaq'].unique().tolist()
n_operations = []
lines = []

for i, job in enumerate(jobs):
    line = []
    df_job = alter[alter['CdItem'] == job]
    df_job.sort_values(by='NuEstagio')
    operations = df_job['NuEstagio'].unique()
    n_operations.append(len(operations))
    line.append(len(operations))
    for l in operations:
        df_op = df_job[df_job['NuEstagio'] == l]
        line.append(len(df_op))
        for op in df_op.itertuples():
            line.append(machines.index(op.CdMaq)+1)
            line.append(op.TempoPadrao * 60) # convertendo o tempo de horas para minutos
    line.append('\n')
    lines.append(line)

lines = [[len(jobs), len(machines), sum(n_operations)/len(jobs), '\n']] + lines
lines+= [['\n']]

print(lines)
with open('.\instances_translated\FTQL.fjs', 'w') as f:
    f.writelines([' '.join([str(x) for x in line]) for line in lines])

[[5, 5, 4.8, '\n'], [5, 1, 1, 1440.0, 1, 2, 2880.0, 1, 3, 2880.0, 1, 4, 1440.0, 1, 5, 1440.0, '\n'], [5, 1, 1, 1440.0, 1, 4, 1440.0, 1, 3, 1440.0, 1, 5, 1440.0, 1, 2, 1440.0, '\n'], [5, 1, 1, 1440.0, 1, 5, 2880.0, 1, 2, 4320.0, 1, 3, 1440.0, 1, 4, 1440.0, '\n'], [4, 1, 1, 1440.0, 1, 3, 2880.0, 1, 2, 2880.0, 1, 4, 2880.0, '\n'], [5, 1, 1, 2880.0, 1, 3, 1440.0, 1, 4, 1440.0, 1, 2, 1440.0, 1, 4, 1440.0, '\n'], ['\n']]


C:\Users\arthur.silva\AppData\Local\Temp\ipykernel_13396\2084533360.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  alter = pd.read_sql('select * from ALTERNATIVAS', conn)


In [91]:
alter

,CdItem,NuEstagio,Escopo,CdMaq,CdFerr,TaxaProd,TaxaProdUnid,ConsumoMaq,ClasseMaq,OperadoresNum,TempoPadrao
0,A,1,MAQ,TRACAR,-1,0.041667,Pecas/hora,1.0,1,1.0,24.0
1,A,2,MAQ,TORNEAR,-1,0.020833,Pecas/hora,1.0,1,1.0,48.0
2,A,3,MAQ,MANDRILAR,-1,0.020833,Pecas/hora,1.0,1,1.0,48.0
3,A,4,MAQ,FRESAR,-1,0.041667,Pecas/hora,1.0,1,1.0,24.0
4,A,5,MAQ,FURAR,-1,0.041667,Pecas/hora,1.0,1,1.0,24.0
5,B,1,MAQ,TRACAR,-1,0.041667,Pecas/hora,1.0,1,1.0,24.0
6,B,2,MAQ,FRESAR,-1,0.041667,Pecas/hora,1.0,1,1.0,24.0
7,B,3,MAQ,MANDRILAR,-1,0.041667,Pecas/hora,1.0,1,1.0,24.0
8,B,4,MAQ,FURAR,-1,0.041667,Pecas/hora,1.0,1,1.0,24.0
9,B,5,MAQ,TORNEAR,-1,0.041667,Pecas/hora,1.0,1,1.0,24.0
